### Sentiment anlayis for the Reviews Using VENDER

 

<b> Importing needed Libaries

In [ ]:
!pip install vaderSentiment
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

     |████████████████████████████████| 133kB 18.9MB/s 


<b>Creating Data Frame with the csv file

In [ ]:
filePath = '/content/amazon_review_updated_label.txt'
df = pd.read_csv(filePath, delimiter='\t')
df.head()

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,Fake,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,Fake,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,Fake,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,Fake,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,Fake,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


<b>Using VADER to analayze the sentiment of the reviews


In [ ]:
df = df.drop(['LABEL','DOC_ID', 'RATING', 'VERIFIED_PURCHASE', 'PRODUCT_CATEGORY', 'PRODUCT_ID', 'PRODUCT_TITLE', 'REVIEW_TITLE'], axis="columns")
df

,LABEL,REVIEW_TEXT
0,Fake,"When least you think so, this product will sav..."
1,Fake,Lithium batteries are something new introduced...
2,Fake,I purchased this swing for my baby. She is 6 m...
3,Fake,I was looking for an inexpensive desk calcolat...
4,Fake,I only use it twice a week and the results are...
...,...,...
20995,Real,"I bought these for work. I have high arches, ..."
20996,Real,Crocs are one of only two brands of shoes that...
20997,Real,I love moccasins This fit like it was custom ...
20998,Real,I wish these were a little more durable. I got...


In [ ]:
df = df.drop([''], axis="columns")
df

,REVIEW_TEXT
0,"When least you think so, this product will sav..."
1,Lithium batteries are something new introduced...
2,I purchased this swing for my baby. She is 6 m...
3,I was looking for an inexpensive desk calcolat...
4,I only use it twice a week and the results are...
...,...
20995,"I bought these for work. I have high arches, ..."
20996,Crocs are one of only two brands of shoes that...
20997,I love moccasins This fit like it was custom ...
20998,I wish these were a little more durable. I got...


In [ ]:
# Getting a compound score
def sentimental_compound(sentence):
    analyzer = SentimentIntensityAnalyzer()
    df_sentiment = analyzer.polarity_scores(sentence)
    score = df_sentiment['compound']
    return score

# VADER predicting Positive negative
def sentimental_score(sentence):
    analyzer = SentimentIntensityAnalyzer()
    df_sentiment = analyzer.polarity_scores(sentence)
    score = df_sentiment['compound']
    if score >= 0.5:
        return 'pos'
    elif (score > -0.5) and (score < 0.5) :
        return 'neu'
    elif score <= -0.5:
        return 'neg'
    

### Importing SVM classification

In [ ]:
#!pip3 install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [ ]:
# # selecting 1000 rows to avoid crashing the browser
# df_selected_rows = df.head(1000)
# #df.insert(1, "lab",0)
# df_selected_rows


In [ ]:
# adding sentiment scores for each reviews
df['compound']=df['REVIEW_TEXT'].apply(lambda x: sentimental_score(x))
df.head()


,REVIEW_TEXT,compound
0,"When least you think so, this product will sav...",neu
1,Lithium batteries are something new introduced...,pos
2,I purchased this swing for my baby. She is 6 m...,pos
3,I was looking for an inexpensive desk calcolat...,neu
4,I only use it twice a week and the results are...,pos


In [ ]:
df

,REVIEW_TEXT,compound
0,"When least you think so, this product will sav...",neu
1,Lithium batteries are something new introduced...,pos
2,I purchased this swing for my baby. She is 6 m...,pos
3,I was looking for an inexpensive desk calcolat...,neu
4,I only use it twice a week and the results are...,pos
...,...,...
20995,"I bought these for work. I have high arches, ...",neu
20996,Crocs are one of only two brands of shoes that...,pos
20997,I love moccasins This fit like it was custom ...,pos
20998,I wish these were a little more durable. I got...,pos


In [ ]:
# Using SVM classifier on the train set
from sklearn.svm import LinearSVC
#classifer = LinearSVC(max_iter=df.shape[0])


In [ ]:
tfidf_lr_pipe = Pipeline([('tfidf', TfidfVectorizer(lowercase=False)), ('svm', LinearSVC(max_iter=df.shape[0]))])

In [ ]:
df.head(5)

,REVIEW_TEXT,compound
0,"When least you think so, this product will sav...",neu
1,Lithium batteries are something new introduced...,pos
2,I purchased this swing for my baby. She is 6 m...,pos
3,I was looking for an inexpensive desk calcolat...,neu
4,I only use it twice a week and the results are...,pos


In [ ]:
df['REVIEW_TEXT']=df['REVIEW_TEXT'].apply(str)
df['compound']=df['compound'].apply(str)

In [ ]:
df.head(5)

,REVIEW_TEXT,compound
0,"When least you think so, this product will sav...",neu
1,Lithium batteries are something new introduced...,pos
2,I purchased this swing for my baby. She is 6 m...,pos
3,I was looking for an inexpensive desk calcolat...,neu
4,I only use it twice a week and the results are...,pos


In [ ]:
x = df.iloc[:,0].values
x

In [ ]:
y = df.iloc[:,1].values
y

array(['neu', 'pos', 'pos', ..., 'pos', 'pos', 'pos'], dtype=object)

In [ ]:
# Splitting the train and test data (80/20)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state= 0)

In [ ]:
# Converting the reviews to a matrix of token counts (Bag of words model)
tfidf_lr_pipe.fit(x_train, y_train)

In [ ]:
y_pred = tfidf_lr_pipe.predict(x_test)

In [ ]:
predicted_df = pd.DataFrame({'predict': y_pred, 'actual': y_test})

In [ ]:
predicted_df

In [ ]:
predicted_df.head(100)

2

In [ ]:
from sklearn.metrics import classification_report
# y_unique = y_test.unique()
# mcm = multilabel_confusion_matrix(y_test, y_pred, labels = y_unique)

In [ ]:
from sklearn.metrics import classification_report

print('SVM results')
#target_names = [' class 0(pos)','class 1(neutral)','class 2(neg)']
print(classification_report(y_test, y_pred,  labels=["pos", "neu", "neg"]))

# Accuracy = (TP + TN) / (TP + TN + FP + FN)
# Precision = (TP) / (TP + FP)
# Recall = (TP) / (TP + FN)
# F1 Score = (2 * Precision * Recall) / (Precision + Recall)

In [ ]:
# Confusion matrix to see the number of true positive, true negative, false positive and false negative
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test, y_pred, labels=["pos", "neu", "neg"])


In [ ]:
import pickle
filename = "/content/serialize_sentiment_model.pkl"  

with open(filename, 'wb') as file:  
    pickle.dump(tfidf_lr_pipe, file)